In speech processing we may use really huge networks cause majority of the processing bottleneck is on CPU and running the CTC function...

Also, CLARIN mobile corpus seems to be too small for advanced network

In [6]:
#!/usr/bin/python3

# For demonstration purposes - Paweł Tomasik
# for CLARIN_MOBILE - generally it is unnormalized

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from keras.models import Model
from keras.layers import LSTM, Conv1D, Dropout, LeakyReLU, Dense, Input, Lambda, TimeDistributed, Flatten, Conv2D, BatchNormalization, GRU, Bidirectional
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.initializers import Orthogonal
from keras.callbacks import Callback

from keras.backend import ctc_batch_cost, expand_dims
import keras.backend as K
import tensorflow as tf

import editdistance  # For digit error rate
import keras
import librosa
import numpy as np
import os

import gc
X = Y = None
gc.collect()

LENGTH = 1700
RECS = 1384 * 4
TRANSL = 307

X = np.zeros([RECS, LENGTH, 24], np.float32)
Y = np.zeros([RECS, TRANSL], np.int16)
counter = 0

for i in range(4):
    Xpart = np.load("datasets/clarin-long/data/clarin-mfcc-rec-aligned-PLC-{}.npy".format(i))
    Ypart = np.load("datasets/clarin-long/data/clarin-mfcc-trans-aligned-PLC-{}.npy".format(i))
    recs = Xpart.shape[0]
    reclen = Xpart.shape[1]
    translen = Ypart.shape[1]
    print(Xpart.shape, Ypart.shape)
    X[counter : counter + recs, :reclen, :] = Xpart
    Y[counter : counter + recs, :translen] = Ypart
    counter += recs
    
print(counter, RECS)
counter //= 32
counter *= 32

X = X[:counter]
Y = Y[:counter]

Xpart.shape

#X = np.clip(X, -3, 3)
MEAN = X.mean()
STD = X.std()

valid = np.random.random(X.shape[0]) > 0.9

(1377, 1109, 24) (1377, 158)
(1382, 1302, 24) (1382, 183)
(1381, 1456, 24) (1381, 198)
(1383, 1603, 24) (1383, 232)
5523 5536


In [7]:
import matplotlib.pyplot as plt
first = lambda x: len(x) - np.where(x[::-1].cumsum())[0][0]
first(X[0])

-13204

In [8]:
mc = keras.callbacks.ModelCheckpoint('models/mfcc-ctc-{epoch:08d}-bigger.h5', 
                                     save_weights_only=False, period=5)

from keras.constraints import max_norm

class MyCallback(keras.callbacks.Callback):
    def __init__(self, fname):
        self.fname = fname
        super(MyCallback, self).__init__()
        
    def on_epoch_end(self, epoch, logs={}):
        content = "Epoch: {}, loss: {}\n".format(epoch, logs.get('loss'))
        with open(self.fname, "a") as f:
            f.write(content)

class StopOnConvergence(Callback):
    def __init__(self, max_repetitions=10):
        super().__init__()
        self.max_repetitions = max_repetitions

    def on_train_begin(self, logs=None):
        self.repetitions = 0
        self.last_loss = np.inf

    def on_epoch_end(self, batch, logs=None):
        logs = logs or {}
        loss = logs.get('val_loss')
        if loss is not None:
            if loss > self.last_loss:
                self.repetitions += 1
            else:
                self.last_loss = loss
                self.repetitions = 0
            if self.repetitions > self.max_repetitions:
                self.model.stop_training = True

            
logger = MyCallback("./training-bigger-log.txt")
NFEATS = 24

def mk_model(max_label_length):
    feature_input = Input(shape = (None, NFEATS))
    layer = Lambda(lambda x: (x - MEAN) / STD)(feature_input)
    layer = Lambda(K.expand_dims)(layer)
    layer_1 = Conv2D(12, [5,1], activation='linear', strides=(2,1), kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer_1 = Conv2D(16, [5,1], activation='linear', strides=(2,1), kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer = TimeDistributed(Flatten())(layer)
    layer_1 = Conv1D(512, 5, activation='linear', kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    for i in range(7):
        layer = Bidirectional(GRU(256, return_sequences = True, recurrent_dropout=0.01, kernel_initializer=Orthogonal(), activation='linear'))(layer)
        layer = LeakyReLU(0.01)(layer)
        layer = BatchNormalization()(layer)
    layer = Dense(1024)(layer)
    layer = LeakyReLU(0.01)(layer)
    layer = BatchNormalization()(layer)
    layer_15 = Dense(NPHONES + 1, activation = 'softmax')(layer)
    label_input = Input(shape = (max_label_length,))
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    loss_lambda = Lambda(lambda args:ctc_batch_cost(*args), output_shape=(1,), name='ctc')([label_input, layer_15, input_length, label_length])
    model = Model([feature_input, label_input, input_length, label_length], [loss_lambda])
    model.summary()
    predictive = Model(feature_input, layer_15)
    return model, predictive

def train(model, trainX, trainy, trainXl, trainyl, epochs = 50):
    # important: batch_size=1 bugs Tensorflow
    optimizer = Adam(0.0003, clipnorm=1.)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    return model.fit([trainX[~valid], trainy[~valid], trainXl[~valid], trainyl[~valid]], np.zeros(trainX[~valid].shape[0]), epochs = epochs,
                     batch_size = 32, callbacks=[mc, logger, StopOnConvergence(4)], validation_data=[[trainX[valid], trainy[valid], trainXl[valid], trainyl[valid]], np.zeros(trainX[valid].shape[0])])

def validate(predictions, valid_length, groundtruth, target_length):
    predictions = keras.backend.ctc_decode(predictions, valid_length, False, 1)
    predictions = predictions[0][0].eval(session=keras.backend.get_session())
    DERs = []
    for index in range(predictions.shape[0]):
        dist = float(editdistance.eval(
            [x for x in predictions[index, :] if x != -1],
            [x for x in groundtruth[index, :] if x != NPHONES]))
        DER = dist / target_length[index]
        DERs.append((DER, target_length[index]))
    return DERs

def try_else(exp, exp_else):
    try:
        return exp()
    except:
        return exp_else

if __name__=='__main__':
    data = X, Y
    NPHONES = Y.max()
    NFEATS = data[0].shape[2]
    X_lens = np.array([try_else(
            (lambda:np.where((x).mean(1) == (x).std(1))[0][0]),
            X.shape[1])
        for x in X])
    X_lens = np.ceil(X_lens / 4.0)
    Y_lens = np.array([np.where(x == NPHONES)[0] for x in data[1]])
    Y_lens = np.array([x[0] if len(x) else 0 for x in Y_lens])
    print(X.shape, Y.shape, X_lens.shape, Y_lens.shape)
    data = data[0][np.where(Y_lens)], data[1][np.where(Y_lens)], X_lens[np.where(Y_lens)].reshape(-1, 1), Y_lens[np.where(Y_lens)].reshape(-1, 1)
    print(list(map(lambda x:x.shape, data)))
    trn, predict = mk_model(data[1].shape[1])
    train(trn, *data, epochs=100) # at 300 it makes sensible predictions

(5504, 1700, 24) (5504, 307) (5504,) (5504,)
[(5504, 1700, 24), (5504, 307), (5504, 1), (5504, 1)]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 24)     0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 24)     0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, None, 24, 1)  0           lambda_1[0][0]                   
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, 24, 12) 72          lambda_2[0][0]                   
__________

Train on 4950 samples, validate on 554 samples
Epoch 1/100
4950/4950 [==============================] - 865s 175ms/step - loss: 363.8622 - val_loss: 348.6704
Epoch 2/100
4950/4950 [==============================] - 850s 172ms/step - loss: 338.3101 - val_loss: 350.7985
Epoch 3/100
4950/4950 [==============================] - 851s 172ms/step - loss: 333.2461 - val_loss: 340.3281
Epoch 4/100
4950/4950 [==============================] - 851s 172ms/step - loss: 321.9867 - val_loss: 337.1284
Epoch 5/100
4950/4950 [==============================] - 850s 172ms/step - loss: 318.2002 - val_loss: 321.3997
Epoch 6/100
4950/4950 [==============================] - 851s 172ms/step - loss: 316.8342 - val_loss: 331.5470
Epoch 7/100
4950/4950 [==============================] - 851s 172ms/step - loss: 316.3010 - val_loss: 343.2165
Epoch 8/100
4950/4950 [==============================] - 850s 172ms/step - loss: 318.1030 - val_loss: nan
Epoch 9/100
4950/4950 [==============================] - 848s 171ms/st

KeyboardInterrupt: 

In [ ]:
pr = predict.predict(X[:1])
pr = K.ctc_decode(pr, X_lens[:1])[0][0]
pr.eval(session=K.get_session())

In [ ]:
Y[:1]

In [ ]:
import editdistance
editdistance.eval(list(pr.eval(session=K.get_session())[0]), list(Y[0, :Y_lens[0]])) / Y_lens[0]

In [ ]:
pr = predict.predict(X[:160])
pr = K.ctc_decode(pr, X_lens[:160])[0][0]
hypos = [list(x) for x in pr.eval(session=K.get_session())]
lens = [x.index(-1) if -1 in x else len(x) for x in hypos]
hypos = [x[:lim] for x, lim in zip(hypos, lens)]
gts = [list(x) for x in Y[:160]]
lens = [x.index(37) if 37 in x else len(x) for x in gts]
gts = [x[:lim] for x, lim in zip(gts, lens)]
[editdistance.eval(gt, hypo) / len(gt) for gt, hypo in zip(gts, hypos)]

In [ ]:
np.array(_).mean()

In [9]:
K.zeros([2]).shape.as_list() == [2]

True

In [18]:
trn.optimizer.weights

[<tf.Variable 'AdamScale/iterations:0' shape=() dtype=int64_ref>,
 <tf.Variable 'training/AdamScale/Variable:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_1:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_2:0' shape=(5, 1, 1, 12) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_3:0' shape=(12,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_4:0' shape=(12,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_5:0' shape=(12,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_6:0' shape=(5, 1, 12, 16) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_7:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_8:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_9:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'training/AdamScale/Variable_10:0' shape=(5, 4112, 512) dtype=float32_ref>,
 <tf.Variable 'training/AdamSca

In [10]:
K.variable([9]).eval(session=K.get_session())

array([9.], dtype=float32)

In [ ]:
predict.save("models/czt-stabilized.h5")

In [1]:
from keras.models import Model
from keras.layers import LSTM, Conv1D, Dropout, LeakyReLU, Dense, Input, Lambda, TimeDistributed, Flatten, Conv2D, BatchNormalization, GRU, Bidirectional
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.initializers import Orthogonal
from keras.callbacks import Callback

from keras.backend import ctc_batch_cost, expand_dims
import keras.backend as K


/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [38]:
a = np.array([[1,2,1.1,2.1,3.3],[1.5,-2,1.12,2.12,-30]])
np.linalg.det(a.T @ a)

-2.9425586192663785e-42